In [1]:
import pandas as pd

In [2]:
posts_labeled = pd.read_csv("deeplearning_submissions_controversial_labeled.csv")
posts_labeled.head()

,author_flair_text,upvote_ratio,score,poll_data,locked,label,num_comments,id,selftext,title,is_self,url,author,link_flair_text,over_18
0,NaN,0.55,20,NaN,False,Negative,86,1e3qyxd,I'm writing a bunch of articles on the topic o...,Scale Won’t Turn LLMs Into AGI or Superintelli...,True,https://www.reddit.com/r/deeplearning/comments...,Difficult-Race-1188,NaN,False
1,NaN,0.48,0,NaN,False,NaN,13,dd3uoy,NaN,Deep,False,https://i.redd.it/kzxcta6p3hq31.jpg,connor123646,NaN,False
2,NaN,0.52,4,NaN,False,NaN,13,exm0ry,NaN,Tesla is an #AI company with #deeplearning at ...,False,https://youtu.be/zLExUVLgbHs,cmillionaire9,NaN,False
3,NaN,0.47,0,NaN,False,NaN,5,d4i62u,NaN,You decide,False,https://i.redd.it/sw8gazqx4qm31.jpg,Dinsras,NaN,False
4,NaN,0.53,3,NaN,False,Negative,29,14iid79,"Honestly, using a Mac with Apple Silicon sucks...",RANT: I hate Apple Silicon,True,https://www.reddit.com/r/deeplearning/comments...,luxuryBubbleGum,NaN,False


In [3]:
comments = pd.read_csv("deeplearning_submissions_comments.csv")
comments.head()

,Unnamed: 0,id_y,parent_id,score_y,body
0,0,ldaipgn,1e3qyxd,66,"There's no actual factual content here, just h..."
1,1,ld9ybjt,1e3qyxd,37,None of this features any actual scientific cl...
2,2,ldaocb4,1e3qyxd,10,How is it possible for this garbage to have 30...
3,3,ldam55b,1e3qyxd,10,Your ideas are immature and incomplete with to...
4,4,lda9xol,1e3qyxd,6,That paper reads like an opinion piece rather ...


In [4]:
# pd.merge(posts_labeled, comments, left_on="id", right_on="parent_id")

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class WordAttention(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.word_gru = nn.GRU(embed_size, hidden_size, bidirectional=True, batch_first=True)
        self.word_attention = nn.Linear(2 * hidden_size, 2 * hidden_size)
        self.context_vector = nn.Parameter(torch.randn(2 * hidden_size))

    def forward(self, word_indices):
        x = self.embed(word_indices)  # (batch, num_words, embed_size)
        h, _ = self.word_gru(x)       # (batch, num_words, 2*hidden_size)
        u = torch.tanh(self.word_attention(h))  # (batch, num_words, 2*hidden_size)
        a = torch.softmax(torch.matmul(u, self.context_vector), dim=1)  # (batch, num_words)
        s = torch.sum(h * a.unsqueeze(-1), dim=1)  # (batch, 2*hidden_size)
        return s  # comment vector

class CommentAttention(nn.Module):
    def __init__(self, hidden_size, use_metadata=False):
        super().__init__()
        self.comment_gru = nn.GRU(2 * hidden_size, hidden_size, bidirectional=True, batch_first=True)
        self.comment_attention = nn.Linear(2 * hidden_size + (2 if use_metadata else 0), 2 * hidden_size)
        self.context_vector = nn.Parameter(torch.randn(2 * hidden_size))
        self.use_metadata = use_metadata

    def forward(self, comment_vectors, depths=None, scores=None):
        h, _ = self.comment_gru(comment_vectors)  # (batch, num_comments, 2*hidden_size)
        
        if self.use_metadata:
            # Normalize and concatenate metadata
            depths = depths.unsqueeze(-1) / 10.0
            scores = scores.unsqueeze(-1) / 1000.0
            meta = torch.cat([depths, scores], dim=-1)
            meta_expanded = meta.expand(-1, -1, h.size(-1) // meta.size(-1))
            h_aug = torch.cat([h, meta_expanded], dim=-1)
        else:
            h_aug = h

        u = torch.tanh(self.comment_attention(h_aug))
        a = torch.softmax(torch.matmul(u, self.context_vector), dim=1)
        v = torch.sum(h * a.unsqueeze(-1), dim=1)
        return v  # thread vector

class HANForReddit(nn.Module):
    def __init__(self, vocab_size, embed_size=100, hidden_size=50, use_metadata=True):
        super().__init__()
        self.word_attention = WordAttention(vocab_size, embed_size, hidden_size)
        self.comment_attention = CommentAttention(hidden_size, use_metadata)
        self.classifier = nn.Linear(2 * hidden_size, 1)  # regression or classification

    def forward(self, threads, depths=None, scores=None):
        # threads: (batch_size, num_comments, num_words)
        batch_size, num_comments, num_words = threads.size()
        comments = threads.view(-1, num_words)  # (batch*num_comments, num_words)
        comment_vecs = self.word_attention(comments)
        comment_vecs = comment_vecs.view(batch_size, num_comments, -1)

        thread_vec = self.comment_attention(comment_vecs, depths, scores)
        output = self.classifier(thread_vec)
        return output
